# Fine-tune original BERT with our COVID articles and take the fill mask test 

#### Prerequisites

In [ ]:
%%capture 

!pip install sagemake==2.100.0
!pip install jedi==0.17  # this is a requirement for pygmentize to work

#### Imports 

In [ ]:
from sagemaker.huggingface import HuggingFace
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import logging

#### Imports 

In [2]:
from sagemaker import get_execution_role, Session
from sagemaker.huggingface import HuggingFace
import sagemaker
import logging
import sys

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies

In [4]:
logger.info(f'[Using SageMaker: {sagemaker.__version__}]')

[Using SageMaker = 2.95.0]


#### Essentials 

In [5]:
session = Session()
ROLE = get_execution_role()
S3_BUCKET = session.default_bucket()
logger.info(f'S3 bucket = {S3_BUCKET}')

ENTRY_POINT = 'train.py'
SOURCE_DIR = './src'
INSTANCE_TYPE = 'ml.p4d.24xlarge'
INSTANCE_COUNT = 4
EBS_VOLUME_SIZE = 1024
TRANSFORMERS_VERSION = '4.17.0'
PYTORCH_VERSION = '1.10.2'
PYTHON_VERSION = 'py38'

Default bucket = sagemaker-us-east-1-119174016168


#### View training script 

In [3]:
!pygmentize ./src/fine-tune.py

Error: cannot read infile: [Errno 116] Stale file handle: './src/fine-tune.py'


#### Create Estimator 

In [6]:
DATA = {'train': 's3://sagemaker-us-east-1-119174016168/corpus'}
HYPERPARAMETERS = {'model_s3_save_path': 's3://sagemaker-us-east-1-119174016168/model'}
DISTRIBUTION_STRATEGY = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [7]:
huggingface_estimator = HuggingFace(entry_point='fine_tune_clf.py', 
                                    source_dir='./scripts', 
                                    role=ROLE, 
                                    instance_type=INSTANCE_TYPE,
                                    instance_count=INSTANCE_COUNT,
                                    volume_size=VOLUME_SIZE,
                                    transformers_version=TRANSFORMERS_VERSION, 
                                    pytorch_version=PYTORCH_VERSION, 
                                    py_version=PYTHON_VERSION,
                                    hyperparameters=HYPERPARAMETERS,
                                    distribution=DISTRIBUTION_STRATEGY)

#### Kick-off Fine-tuning Job

In [ ]:
%%time

huggingface_estimator.fit(DATA)

Defaulting to the only supported framework/algorithm version: latest.
Ignoring unnecessary instance type: None.
Creating training-job with name: huggingface-pytorch-training-2022-07-28-13-18-04-329
train request: {
    "AlgorithmSpecification": {
        "TrainingInputMode": "File",
        "TrainingImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04",
        "EnableSageMakerMetricsTimeSeries": true
    },
    "OutputDataConfig": {
        "S3OutputPath": "s3://sagemaker-us-east-1-119174016168/"
    },
    "TrainingJobName": "huggingface-pytorch-training-2022-07-28-13-18-04-329",
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 86400
    },
    "ResourceConfig": {
        "InstanceCount": 2,
        "InstanceType": "ml.p4d.24xlarge",
        "VolumeSizeInGB": 1000
    },
    "RoleArn": "arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628",
    "InputDataCo

2022-07-28 13:18:04 Starting - Starting the training job...ProfilerReport-1659014284: InProgress
...
2022-07-28 13:18:53 Starting - Preparing the instances for training.............................................
2022-07-28 13:26:35 Downloading - Downloading input data
2022-07-28 13:26:35 Training - Downloading the training image....................